In [1]:
import numpy as np
import pickle
import time
import cv2
import os

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [12]:
%load_ext tensorboard

In [13]:
X = pickle.load(open("X.pickle",'rb'))
y = pickle.load(open("y.pickle",'rb'))

X = X / 255.0

In [14]:
dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]

In [15]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(64, (3,3), input_shape=X.shape[1:], activation='relu'))
            model.add(MaxPooling2D((2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(64, (3,3), activation='relu'))
                model.add(MaxPooling2D((2,2)))

            model.add(Flatten())

            for l in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))


            model.add(Dense(1, activation='sigmoid'))

            model.compile(loss="binary_crossentropy",
                          optimizer="adam",
                         metrics=['accuracy'])


            model.fit(X, y, batch_size=32, validation_split=0.3, epochs=10, callbacks=[tensorboard])

3-conv-64-nodes-1-dense1579719052


W0123 00:20:53.041810 4405083584 deprecation.py:323] From /Users/spiedy/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 179s 10ms/sample - loss: 0.6536 - accuracy: 0.6048 - val_loss: 0.6337 - val_accuracy: 0.6480
Epoch 2/10
17462/17462 [==============================] - 162s 9ms/sample - loss: 0.5500 - accuracy: 0.7150 - val_loss: 0.5416 - val_accuracy: 0.7254
Epoch 3/10
17462/17462 [==============================] - 197s 11ms/sample - loss: 0.4773 - accuracy: 0.7736 - val_loss: 0.4835 - val_accuracy: 0.7695
Epoch 4/10
17462/17462 [==============================] - 205s 12ms/sample - loss: 0.4354 - accuracy: 0.7980 - val_loss: 0.4437 - val_accuracy: 0.7962
Epoch 5/10
17462/17462 [==============================] - 209s 12ms/sample - loss: 0.3927 - accuracy: 0.8237 - val_loss: 0.4396 - val_accuracy: 0.8052
Epoch 6/10
17462/17462 [==============================] - 213s 12ms/sample - loss: 0.3559 - accuracy: 0.8434 - val_loss: 0.4400 - val_accuracy: 0.8045
Epoch 7/10
17462/17462 [======================

In [16]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2869), started 6:16:52 ago. (Use '!kill 2869' to kill it.)

In [13]:
model.save('64x3-CNN.model')

W0122 19:06:01.920569 4533693888 deprecation.py:506] From /Users/spiedy/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
new_model = tf.keras.models.load_model("64x3-CNN.model")

In [3]:
CATEGORIES = ["Dog", "Cat"]

In [6]:
def prepare_data(img_path):
    IMG_SIZE = 70
    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img_array = img_array / 255.0
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [17]:
pred = model.predict([prepare_data('sofia.jpg')])

In [18]:
pred[0][0]

0.9875185